# Example 2.2: Pegboard Mesh
In this example, we investigate converence with respect to the mesh size.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import puncturedfem as pf

In [ ]:
deg = 2
mesh_size = (4, 4)
radius = 0.25

In [ ]:
quad_dict = pf.get_quad_dict(n=64)

In [ ]:
mesh = pf.meshlib.pegboard(size=mesh_size, radius=radius)

In [ ]:
pf.plot.MeshPlot(mesh.edges).draw()

In [ ]:
global_function_space = pf.GlobalFunctionSpace(mesh, deg, quad_dict)

In [ ]:
a = 1.0
c = 1.0
f = pf.Polynomial([(1.0, 0, 0)])

B = pf.BilinearForm(
    diffusion_constant=a,
    reaction_constant=c,
    rhs_poly=f,
)

print(B)

In [ ]:
solver = pf.Solver(
    global_function_space, B, compute_interior_values=False, verbose=False
)
solver.solve()
x = solver.soln

In [ ]:
A = solver.glob_mat.todense()
cond_num = np.linalg.cond(A)

## Examine the finite element matrix (reduced)

In [ ]:
# hack to identify global indices of functions supported on the boundary
non_boudary_indices = set()
for abs_cell_idx in range(global_function_space.mesh.num_cells):
    for key in global_function_space.cell_dofs[abs_cell_idx]:
        if not key.is_on_boundary:
            non_boudary_indices.add(key.glob_idx)
non_boudary_indices = sorted(non_boudary_indices)

# reduced finite element matrix (with boundary functions removed)
A_reduced = A[non_boudary_indices, :][:, non_boudary_indices]
cond_num_reduced = np.linalg.cond(A_reduced)
dof_reduced = A_reduced.shape[0]
print(f"cond_num = {cond_num_reduced:.2e}")

# eigenvalues
eigenvalues_reduced = np.linalg.eigvalsh(A_reduced)
plt.figure()
plt.semilogy(eigenvalues_reduced, "ko-")
plt.grid(True)

## $D^{-1/2} A D^{-1/2}$ preconditioner

In [ ]:
# preconditioned reduced system
D = np.diag(A_reduced)

tol = 1e-6
zeros = np.where(D < tol)[0]
print(f"number of zeros = {len(zeros)}")
print(f"zero indices = {zeros}")
print(f"zero values = {D[zeros]}")

plt.figure()
D_sorted = np.sort(D)
plt.semilogy(D_sorted, "bo-")
plt.grid(True)
plt.title("Diagonal of A_reduced (sorted)")
plt.show()

D_sqrt = np.diag(np.sqrt(D))
D_sqrt_inv = np.linalg.inv(D_sqrt)
A_reduced_precond = D_sqrt_inv @ A_reduced @ D_sqrt_inv
cond_num_reduced_precond = np.linalg.cond(A_reduced_precond)
print(f"cond_num = {cond_num_reduced_precond:.2e}")

eigenvalues_reduced_precond = np.linalg.eigvalsh(A_reduced_precond)
plt.figure()
plt.semilogy(eigenvalues_reduced_precond, "ko-")
plt.grid(True)

## $H^1$ norm of the error

In [ ]:
A = solver.stiff_mat
M = solver.mass_mat

h1_sq_norm = x @ (A + M) @ x
h1_sq_norm_exact = 0.033523205539

error_norm = np.sqrt(h1_sq_norm_exact - h1_sq_norm)

In [ ]:
print(f"poly_deg = {deg}")
print(f"mesh_size = {mesh_size}")
print(f"dof = {dof_reduced}")
print(f"cond_num = {cond_num_reduced:.2e}")
print(f"H1 norm of error: {error_norm:.2e}")

| p | (m,n)   | dof      | cond(A)  | H1 Error | Ratio |
|---|---------|----------|----------|----------|-------|
| 1 | (2,2)   | 13       | 1.38e+01 | 7.01e-02 | n/a   |
| 1 | (4,4)   | 57       | 4.18e+01 | 3.88e-02 | 1.81  |
| 1 | (8,8)   | 241      | 1.57e+02 | 1.99e-02 | 1.95  |
| 1 | (16,16) | 993      | 6.20e+02 | 1.00e-02 | 1.99  |
| 1 | (32,32) | 4033     | 2.47e+03 | 5.03e-03 | 1.99  |
|---|---------|----------|----------|----------|-------|
| 2 | (2,2)   | 33       | 1.41e+02 | 6.53e-03 | n/a   |
| 2 | (4,4)   | 145      | 7.89e+02 | 2.55e-03 | 2.56  |
| 2 | (8,8)   | 609      | 2.16e+04 | 8.04e-04 | 3.17  |
| 2 | (16,16) | 2497     | 1.27e+06 | 2.33e-04 | 3.45  |
| 2 | (32,32) | 10113    | 8.04e+07 | 6.42e-05 | 3.63  |
|---|---------|----------|----------|----------|-------|
| 3 | (2,2)   | 61       | 6.63e+03 | 8.79e-04 | n/a   |
| 3 | (4,4)   | 265      | 1.32e+04 | 1.82e-04 | 4.83  |
| 3 | (8,8)   | 1105     | 5.28e+04 | 4.06e-05 | 4.48  |
<!-- | 3 | (16,16) | 4531     | 4.42e+18 | 7.81e-06 | 5.20  | -->

In [ ]:
errors1 = [7.01e-02, 3.88e-02, 1.99e-02, 1.00e-02, 5.03e-03]
errors2 = [6.53e-03, 2.55e-03, 8.04e-04, 2.33e-04, 6.42e-05]
errors3 = [8.79e-04, 1.82e-04, 4.06e-05, 7.81e-06]

for errors in [errors1, errors2, errors3]:
    print("")
    for i in range(1, len(errors)):
        print(f"{errors[i-1]/errors[i]:.2f}")